# Notebook I: Cleaning and EDA #

In [1]:
import pandas as pd

##### In this notebook, I will be cleaning the data and performing exploratory analysis to better understand what information our app needs to incorporate and how it informs the design of the app. #####

In [2]:
sales = pd.read_csv('Trans_TWSupport Sale w. Cash Discount (v15)_1_935.csv')

In [3]:
sales.head()

,startDate,endDate,Date,gps x,gps y,Mobile User,Non achtè / Buyer's First Name,Siyati achtè a / Buyer's Last Name,Telefon Natcom achtè a / Buyer's Natcom Phone Number,Buyer's Digicel Phone Number / Buyer's Digicel Phone Number,...,Kontra Foto,Nimewo Kontra,Kod pwodwi a,Rabè Lajan Kach / Cash Discount,Èske ou pral peye an plen kounye a? / Are you going to pay in full now?,Loan Name,Kantite lajan Depo Espere / Deposit Amount Expected,montan depoze / Deposit Amount,Peman espere / Payment Expected,Kantite lajan Peman Lajan Kach / Cash Payment Amount
0,startDate,endDate,Date,gps x,gps y,Mobile User,non_acht,siyati_acht_a,telefon_natcom_acht_a,buyers_natcom_phone_number_buyers_digice,...,kontra_foto,nimewo_nontra_1,kod_pwodwi_a,cash_discount_1,are_you_going_to_pay_in_full_now,loan_name,deposit_amount_expected,montan_depoze_deposit_amount,payment_expected,cash_payment_amount
1,2022-01-12 11:06:28-0600,2022-01-12 11:06:45-0600,2022-01-12,NaN,NaN,UC TaroWorks Admin,S,S,NaN,NaN,...,NaN,NaN,NaN,"Si ou peye an plen lajan kach, ou pral resevwa...",Wi / Yes,S - Antreprenè Sistèm Filtraj ak Pye / Entrepr...,NaN,NaN,2900.0,2900.0
2,2022-01-12 11:08:42-0600,2022-01-12 11:10:28-0600,2022-01-12,NaN,NaN,Fenson Cherenfant,Markenson,Toussaint,40573348,NaN,...,https://unlockingcommunities.my.salesforce.com...,01627,6473,"Si ou peye an plen lajan kach, ou pral resevwa...",Wi / Yes,Toussaint - Antreprenè Sistèm Filtraj ak Pye ...,NaN,NaN,2900.0,2900.0
3,2022-01-12 11:15:31-0600,2022-01-12 11:17:01-0600,2022-01-12,NaN,NaN,Fenson Cherenfant,Ghislaine,Myrthil,43564343,NaN,...,https://unlockingcommunities.my.salesforce.com...,01625,6427,"Si ou peye an plen lajan kach, ou pral resevwa...",Wi / Yes,Myrthil - Antreprenè Sistèm Filtraj ak Pye / E...,NaN,NaN,2900.0,2900.0
4,2022-01-12 11:17:45-0600,2022-01-12 11:19:08-0600,2022-01-12,NaN,NaN,Fenson Cherenfant,Nastal,Gabriel,35235271,NaN,...,https://unlockingcommunities.my.salesforce.com...,01626,6426,"Si ou peye an plen lajan kach, ou pral resevwa...",Wi / Yes,Gabriel - Antreprenè Sistèm Filtraj ak Pye / E...,NaN,NaN,2900.0,2900.0


The first row is a secondary column header, so it will be dropped.

In [4]:
sales.drop(0, inplace=True)

In [5]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 935 entries, 1 to 935
Data columns (total 44 columns):
 #   Column                                                                           Non-Null Count  Dtype 
---  ------                                                                           --------------  ----- 
 0   startDate                                                                        935 non-null    object
 1   endDate                                                                          935 non-null    object
 2   Date                                                                             935 non-null    object
 3   gps x                                                                            0 non-null      object
 4   gps y                                                                            0 non-null      object
 5   Mobile User                                                                      935 non-null    object
 6   Non achtè / Buyer'

startDate, endDate, and Date seem rather repetitive.  The duration always seems to be a few minutes.

There is also a "Date of Sale" field, which I will compare with the Date field to see if they're different:

In [6]:
for times in list(zip(sales['Date'], sales['Dat Vann / Date of Sale']))[:5]:
    print(times)

('2022-01-12', '2022-01-12')
('2022-01-12', '2022-01-08')
('2022-01-12', '2022-01-08')
('2022-01-12', '2022-01-08')
('2022-01-12', '2022-01-08')


There are clearly differences between "Date" and "Date of Sale."  I don't know for sure why, but I suspect "Date," "startDate," and "endDate" all apply to when the entry was recorded in the app.  For the purpose of identifyng patterns, "Sale Date" should be more useful.

gps x and gps y are always Null and will also be dropped.

In [7]:
sales.drop(['gps x', 'gps y', 'startDate', 'endDate', 'Date'], axis=1, inplace=True)

Now convert Sale Date to datetime format and rename it.

I'm also converting "Expected Delivery Date" while I'm at it.

In [8]:
sales['sale_date'] = pd.to_datetime(sales['Dat Vann / Date of Sale'])
sales['exp_delivery_date'] = pd.to_datetime(sales['Expected Delivery Date'])
sales.drop(['Dat Vann / Date of Sale', 'Expected Delivery Date'], axis=1, inplace=True)

In "Why are you purchasing the product?" it's common for more than one reason to be given.  It will be best to One Hot encode this.

In [9]:
reasons = sales['Poukisa w’ap achte filtè sa a? / Why are you buying the product?']

In [10]:
options = set()
for reason in reasons:
    for option in reason.split(';'):
        if option not in options:
            options.add(option)

In [11]:
options

{'Lòt',
 'Paske mwen pa renmen gou klò a',
 'Paske pitit mwen yo ap vin malad',
 'Paske pwodwi a pi bon mache pase sa mwen konn itilize'}

"Lòt" means "Other"

"Paske mwen pa renmen gou klò a" means "Because I don't like the taste of chlorine"

"Paske pitit mwen yo ap vin malad" means "Because my children are getting sick"

"Paske pwodwi a pi bon mache pase sa mwen konn itilize" means "Because the product is cheaper than I used to"

"kisa_acht_a_espere_kap_rive_l_li_achte_p" is just a secondary column title

In [12]:
other = []
chlorine = []
sick = []
cheaper = []

for reason in reasons:
    if 'Lòt' in reason.split(';'):
        other.append(1)
    else:
        other.append(0)
    if 'Paske mwen pa renmen gou klò a' in reason.split(';'):
        chlorine.append(1)
    else:
        chlorine.append(0)
    if 'Paske pitit mwen yo ap vin malad' in reason.split(';'):
        sick.append(1)
    else:
        sick.append(0)
    if 'Paske pwodwi a pi bon mache pase sa mwen konn itilize' in reason.split(';'):
        cheaper.append(1)
    else:
        cheaper.append(0)

sales['reason_other'] = other
sales['reason_chlorine'] = chlorine
sales['reason_sick'] = sick
sales['reason_cheaper'] = cheaper

sales.drop(['Poukisa w’ap achte filtè sa a? / Why are you buying the product?'], axis=1, inplace=True)

Other columns that don't seem useful for EDA:

- Mobile User
- Location ID (always the same - see below)
- Order Type (always the same)
- Currency (always the same)
- Direct Order (always the same)
- Status (always the same)
- Buyer Record Type (always the same)
- Kontra Foto (seems to mean contract photo, which is currently a URL I can't access)
- Nimewo Kontra (contract number, not useful in this analysis)
- Unit price (always 0)

In [13]:
sales['Location ID'].value_counts()

LOC-0002    935
Name: Location ID, dtype: int64

Dropping the columns listed above:

In [14]:
sales['Order Type'].value_counts()

Sales Order    935
Name: Order Type, dtype: int64

In [15]:
sales['Currency'].value_counts()

HTG    935
Name: Currency, dtype: int64

In [16]:
sales['Direct Order'].value_counts()

Yes    935
Name: Direct Order, dtype: int64

In [17]:
sales['Status'].value_counts()

Completed    935
Name: Status, dtype: int64

In [18]:
sales['Buyer Record Type'].value_counts()

Product Owner and In-Country Contact    935
Name: Buyer Record Type, dtype: int64

In [19]:
sales['Unit price'].value_counts()

0    935
Name: Unit price, dtype: int64

In [20]:
sales.drop(['Mobile User', 'Location ID', 'Order Type', 'Currency', 'Direct Order', 'Status',
           'Buyer Record Type', 'Kontra Foto', 'Nimewo Kontra', 'Unit price'], axis=1, inplace=True)

The following columns remain in the data frame:

In [21]:
sales.columns

Index(['Non achtè / Buyer's First Name', 'Siyati achtè a / Buyer's Last Name',
       'Telefon Natcom achtè a / Buyer's Natcom Phone Number',
       'Buyer's Digicel Phone Number / Buyer's Digicel Phone Number',
       'Konbyen moun ki abite lakay achtè a / How many people live in the buyers house?',
       'Source Warehouse ID', 'Pricebook ID', 'Product ID', 'Product Name',
       'Quantity', 'Last Qty Delivered', 'Entrepreneur ID', 'Business ID',
       'Expected Deposit Amount', 'Pri pwodwi / Product Price',
       'Kantite lajan rabè lajan kach / Cash Discount Amount',
       'Pri total pwodwi / Total Product Price',
       'Pri Lajan Kach Total / Total Cash Discount Price', 'Kod pwodwi a',
       'Rabè Lajan Kach / Cash Discount',
       'Èske ou pral peye an plen kounye a? / Are you going to pay in full now?',
       'Loan Name', 'Kantite lajan Depo Espere / Deposit Amount Expected',
       'montan depoze / Deposit Amount', 'Peman espere / Payment Expected',
       'Kantite lajan

Renaming the remaining columns:

In [22]:
col_names = {
    "Non achtè / Buyer's First Name" : "buyer_name_first",
    "Siyati achtè a / Buyer's Last Name" : "buyer_name_last",
    "Telefon Natcom achtè a / Buyer's Natcom Phone Number" : "buyer_natcom_phone",
    "Buyer's Digicel Phone Number / Buyer's Digicel Phone Number" : "buyer_digicel_phone",
    "Konbyen moun ki abite lakay achtè a / How many people live in the buyers house?" : "household_size",
    "Source Warehouse ID" : "warehouse_id",
    "Pricebook ID" : "pricebook_id",
    "Product ID" : "product_id",
    "Product Name" : "product_name",
    "Quantity" : "quantity",
    "Last Qty Delivered" : "last_qty_delivered",
    "Entrepreneur ID" : "id_entrepreneur",
    "Business ID" : "id_business",
    "Expected Deposit Amount" : "deposit_expected",
    "Pri pwodwi / Product Price" : "price_product",
    "Kantite lajan rabè lajan kach / Cash Discount Amount" : "cash_discount_amount",
    "Pri total pwodwi / Total Product Price" : "price_product_total",
    "Pri Lajan Kach Total / Total Cash Discount Price" : "price_cash_discount_total",
    "Kod pwodwi a" : "product_code",
    "Rabè Lajan Kach / Cash Discount" : 'cash_discount_description',
    "Èske ou pral peye an plen kounye a? / Are you going to pay in full now?" : "full_pay",
    "Loan Name" : "loan_name",
    "Kantite lajan Depo Espere / Deposit Amount Expected" : "deposit_expected_2",
    "montan depoze / Deposit Amount" : "deposit_amount",
    "Peman espere / Payment Expected" : "expected_payment",
    "Kantite lajan Peman Lajan Kach / Cash Payment Amount" : "cash_payment_amount"
}

sales.rename(col_names, axis=1, inplace=True)

In [23]:
sales.columns

Index(['buyer_name_first', 'buyer_name_last', 'buyer_natcom_phone',
       'buyer_digicel_phone', 'household_size', 'warehouse_id', 'pricebook_id',
       'product_id', 'product_name', 'quantity', 'last_qty_delivered',
       'id_entrepreneur', 'id_business', 'deposit_expected', 'price_product',
       'cash_discount_amount', 'price_product_total',
       'price_cash_discount_total', 'product_code',
       'cash_discount_description', 'full_pay', 'loan_name',
       'deposit_expected_2', 'deposit_amount', 'expected_payment',
       'cash_payment_amount', 'sale_date', 'exp_delivery_date', 'reason_other',
       'reason_chlorine', 'reason_sick', 'reason_cheaper'],
      dtype='object')

Some of these feature names seem to be redundant, so I'm investigating them further.

In [24]:
sales[['product_id', 'product_name', 'product_code']]

,product_id,product_name,product_code
1,P-012,Antreprenè Sistèm Filtraj ak Pye / Entrepreneu...,NaN
2,P-012,Antreprenè Sistèm Filtraj ak Pye / Entrepreneu...,6473
3,P-012,Antreprenè Sistèm Filtraj ak Pye / Entrepreneu...,6427
4,P-012,Antreprenè Sistèm Filtraj ak Pye / Entrepreneu...,6426
5,P-012,Antreprenè Sistèm Filtraj ak Pye / Entrepreneu...,6623
...,...,...,...
931,P-007,Recho / Stove,NaN
932,P-007,Recho / Stove,NaN
933,P-007,Recho / Stove,NaN
934,P-007,Recho / Stove,NaN


In [25]:
sales['product_code'].value_counts()

2207    6
6863    5
6826    4
6513    4
6113    4
       ..
7051    1
5758    1
6458    1
6461    1
6224    1
Name: product_code, Length: 572, dtype: int64

In [26]:
sales[sales['product_code']=='2207']['product_id']

669    P-006
670    P-006
671    P-006
672    P-006
673    P-006
825    P-006
Name: product_id, dtype: object

I'm really not sure what product code means, or why it's different from product ID.  Product code seems to be more specific than product ID, but not all products have a code.

In [27]:
sales['product_id'].value_counts()

P-006    567
P-007    217
P-012    147
P-014      4
Name: product_id, dtype: int64

In [28]:
sales['product_name'].value_counts()

Sistèm Filtraj ak Pye / Filter                            567
Recho / Stove                                             217
Antreprenè Sistèm Filtraj ak Pye / Entrepreneur Filter    147
Filtre / Filter                                             4
Name: product_name, dtype: int64

It appears certain that there is a 1-to-1 relationship between product ID and product name.  It is not clear what the difference is between the three different filters.

In [29]:
sales[['product_id', 'warehouse_id', 'pricebook_id', 'id_entrepreneur', 'id_business']]

,product_id,warehouse_id,pricebook_id,id_entrepreneur,id_business
1,P-012,WH-0335,PB-006,C-1120,A=01321
2,P-012,WH-0531,PB-006,C-1142,A=01336
3,P-012,WH-0531,PB-006,C-1142,A=01336
4,P-012,WH-0531,PB-006,C-1142,A=01336
5,P-012,WH-0531,PB-006,C-1142,A=01336
...,...,...,...,...,...
931,P-007,WH-0118,PB-006,C-656,A=00564
932,P-007,WH-0118,PB-006,C-656,A=00564
933,P-007,WH-0118,PB-006,C-656,A=00564
934,P-007,WH-0118,PB-006,C-5569,A=00564


In [30]:
sales[sales['warehouse_id']=='WH-0531']['product_id']

2      P-012
3      P-012
4      P-012
5      P-012
6      P-012
       ...  
903    P-006
904    P-007
905    P-007
906    P-007
907    P-006
Name: product_id, Length: 173, dtype: object

There is a many-to-many relationship between product ID and warehouse ID, suggesting that the warehouses are not specialized.

In [31]:
sales[sales['id_business']=='A=01336']['id_entrepreneur']

2      C-1142
3      C-1142
4      C-1142
5      C-1142
6      C-3974
        ...  
903    C-1142
904    C-1142
905    C-1142
906    C-3972
907    C-3973
Name: id_entrepreneur, Length: 173, dtype: object

In [32]:
sales[sales['id_entrepreneur']=='C-1142']['id_business'].value_counts()

A=01336    86
Name: id_business, dtype: int64

In [33]:
for entrepreneur in sales['id_entrepreneur'].unique():
    if len(sales[sales['id_entrepreneur']==entrepreneur]['id_business'].value_counts()) != 1:
        print('Found an entrepreneur associated with multiple businesses.')

It appears that each entrepreneur is associated with a single business.

In [34]:
sales['pricebook_id'].value_counts()

PB-006    926
PB-002      9
Name: pricebook_id, dtype: int64

In [35]:
sales[sales['pricebook_id']=='PB-006'][['product_id', 'product_name']].value_counts()

product_id  product_name                                          
P-006       Sistèm Filtraj ak Pye / Filter                            562
P-007       Recho / Stove                                             217
P-012       Antreprenè Sistèm Filtraj ak Pye / Entrepreneur Filter    147
dtype: int64

In [36]:
sales[sales['pricebook_id']=='PB-002'][['product_id', 'product_name']].value_counts()

product_id  product_name                  
P-006       Sistèm Filtraj ak Pye / Filter    5
P-014       Filtre / Filter                   4
dtype: int64

Although there are two different pricebooks, one is used very infrequently.  At least one product appears in both pricebooks.

In [53]:
sales['product_id'].value_counts()

P-006    567
P-007    217
P-012    147
P-014      4
Name: product_id, dtype: int64

In [39]:
sales[['pricebook_id', 'deposit_expected', 'price_product', 'cash_discount_amount',
                                        'price_product_total', 'price_cash_discount_total', 'full_pay', 'deposit_expected_2',
                                        'deposit_amount', 'expected_payment', 'cash_payment_amount']]

,pricebook_id,deposit_expected,price_product,cash_discount_amount,price_product_total,price_cash_discount_total,full_pay,deposit_expected_2,deposit_amount,expected_payment,cash_payment_amount
1,PB-006,2900,2900.0,2900.0,2900.0,2900,Wi / Yes,NaN,NaN,2900.0,2900.0
2,PB-006,2900,2900.0,2900.0,2900.0,2900,Wi / Yes,NaN,NaN,2900.0,2900.0
3,PB-006,2900,2900.0,2900.0,2900.0,2900,Wi / Yes,NaN,NaN,2900.0,2900.0
4,PB-006,2900,2900.0,2900.0,2900.0,2900,Wi / Yes,NaN,NaN,2900.0,2900.0
5,PB-006,2900,2900.0,2900.0,2900.0,2900,Wi / Yes,NaN,NaN,2900.0,2900.0
...,...,...,...,...,...,...,...,...,...,...,...
931,PB-006,1500,3000.0,2500.0,3000.0,2500,Non / No,1500.0,1500,NaN,NaN
932,PB-006,1500,3000.0,2500.0,3000.0,2500,Non / No,1500.0,1500,NaN,NaN
933,PB-006,1500,3000.0,2500.0,3000.0,2500,Non / No,1500.0,1500,NaN,NaN
934,PB-006,1500,3000.0,2500.0,3000.0,2500,Wi / Yes,NaN,NaN,2500.0,2500.0


Based on the above, there is some reduncancy in the way pricing information is being recorded.

Below, I am extracting the price of each product from each pricebook, both for full pay and loan transactions.

In [66]:
sales[(sales['pricebook_id'] == 'PB-006') & (sales['full_pay'] == 'Wi / Yes') & (sales['product_id'] == 'P-006')]['expected_payment'].value_counts()

3500.0     437
7000.0       2
10500.0      2
Name: expected_payment, dtype: int64

In [75]:
sales[(sales['pricebook_id'] == 'PB-006') & (sales['full_pay'] == 'Non / No') & (sales['product_id'] == 'P-006')][['deposit_expected', 'price_product']].value_counts()

deposit_expected  price_product
2050              4100.0           121
dtype: int64

In [72]:
sales[(sales['pricebook_id'] == 'PB-002') & (sales['full_pay'] == 'Wi / Yes') & (sales['product_id'] == 'P-006')]['expected_payment'].value_counts()

4650.0    3
Name: expected_payment, dtype: int64

In [76]:
sales[(sales['pricebook_id'] == 'PB-002') & (sales['full_pay'] == 'Non / No') & (sales['product_id'] == 'P-006')][['deposit_expected', 'price_product']].value_counts()

deposit_expected  price_product
1500              5550.0           2
dtype: int64

In [68]:
sales[(sales['pricebook_id'] == 'PB-006') & (sales['full_pay'] == 'Wi / Yes') & (sales['product_id'] == 'P-012')]['expected_payment'].value_counts()

2900.0    137
Name: expected_payment, dtype: int64

In [77]:
sales[(sales['pricebook_id'] == 'PB-006') & (sales['full_pay'] == 'Non / No') & (sales['product_id'] == 'P-012')][['deposit_expected', 'price_product']].value_counts()

deposit_expected  price_product
2900              2900.0           10
dtype: int64

In [70]:
sales[(sales['pricebook_id'] == 'PB-006') & (sales['full_pay'] == 'Wi / Yes') & (sales['product_id'] == 'P-007')]['expected_payment'].value_counts()

2500.0    142
5000.0      2
7500.0      1
Name: expected_payment, dtype: int64

In [78]:
sales[(sales['pricebook_id'] == 'PB-006') & (sales['full_pay'] == 'Non / No') & (sales['product_id'] == 'P-007')][['deposit_expected', 'price_product']].value_counts()

deposit_expected  price_product
1500              3000.0           72
dtype: int64

In [79]:
sales[(sales['pricebook_id'] == 'PB-002') & (sales['full_pay'] == 'Wi / Yes') & (sales['product_id'] == 'P-014')]['expected_payment'].value_counts()

2500.0    4
Name: expected_payment, dtype: int64

In [80]:
sales[(sales['pricebook_id'] == 'PB-002') & (sales['full_pay'] == 'Non / No') & (sales['product_id'] == 'P-014')][['deposit_expected', 'price_product']].value_counts()

Series([], dtype: int64)